In [1]:
import tkinter
import tkinterdnd2
from tkinterdnd2 import *
from tkinter import *
import tkinter as tk
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image,ImageTk
import PIL
class transform:
    def __init__(self,path,ksize = 7,depth = .7):
        self.path = path
        self.disp_img = PIL.Image.open(path).resize((285,275))
        self.image =  cv2.resize(cv2.imread(path),(285,275))
        self.image_rgb = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.new = None
        self.k_size = ksize
        self.depth = depth
    def convert(self):
        grey_img = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        invert_img=cv2.bitwise_not(grey_img)
        blur_img=cv2.GaussianBlur(invert_img, (self.k_size,self.k_size),0)
        invblur_img=cv2.bitwise_not(blur_img)
        self.new=cv2.divide(grey_img,invblur_img, scale=256.0)
        self.sketch = PIL.Image.fromarray(self.new)
        return self.sketch

    def display(self):
        if self.new == None:
            _ = self.convert()
        fig = plt.figure()
        ax1 = fig.add_subplot(1,2,1)
        ax1.imshow(self.image_rgb)
        ax2 = fig.add_subplot(1,2,2)
        ax2.imshow(self.minimize(self.new), cmap = 'gray')
    def minimize(self,data):
        data = data/data.max()
        data -= data.min()
        return np.clip(data - (data.max()*self.depth),0,1)


In [2]:
import tkinter

class gui_stuff:
    def create_gui(self,):
        self.func = None
        self.ws = TkinterDnD.Tk()
        self.ws.title('SKETCH')
        self.ws.geometry('800x500')
        self.ws.config(bg='#152f31')
        image_entry = tkinter.Label(self.ws, height=20, width=40)
        image_entry.place(relx = .1,rely = .23)
        image_entry.drop_target_register(DND_FILES)
        image_entry.dnd_bind('<<Drop>>', self.display_img)

        sketch_entry = tk.Label(self.ws, height=20, width=40)
        sketch_entry.place(relx = .55,rely = .23)

        converter = tk.Button(self.ws,text = 'CONVERT',height = 1,width = 10 ,command = self.display_sketch)
        converter.place(relx = .45,rely = .85)
        
        self.ws.mainloop()
    def display_img(self,event):
        path = event.data[1:-1]
        self.func = transform(path)
        photo = ImageTk.PhotoImage(self.func.disp_img)
        image_entry = tk.Label(self.ws,image = photo)
        image_entry.image = photo
        image_entry.place(relx = .1,rely = .23)
    def display_sketch(self):
        if self.func != None:
            sketch = ImageTk.PhotoImage(self.func.convert())
            sketch_entry = tk.Label(self.ws,image = sketch)
            sketch_entry.image = sketch
            sketch_entry.place(relx = .55,rely = .23)
            
        else:
            sketch_entry = tk.Label(self.ws,text='⚠️please add an image')
            sketch_entry.place(relx = .55,rely = .23)
gui_stuff().create_gui()  



